# ✋ Hand Gesture Recognition using LeapGestRecog Dataset

Build a gesture-based control system using image classification with SVM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/leapGestRecog.zip"  # Update path if different
extract_path = "/content/data/gestures"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
!pip install opencv-python-headless scikit-learn matplotlib tqdm


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import os

def load_gesture_images(data_dir, img_size=64):
    X = []
    y = []
    label_dict = {}
    label_counter = 0

    for person_folder in sorted(os.listdir(data_dir)):
        person_path = os.path.join(data_dir, person_folder)
        if not os.path.isdir(person_path):
            continue
        for gesture_folder in sorted(os.listdir(person_path)):
            gesture_path = os.path.join(person_path, gesture_folder)
            if not os.path.isdir(gesture_path):
                continue
            label = gesture_folder
            if label not in label_dict:
                label_dict[label] = label_counter
                label_counter += 1
            for img_name in os.listdir(gesture_path):
                img_path = os.path.join(gesture_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv2.resize(img, (img_size, img_size))
                X.append(img.flatten())
                y.append(label_dict[label])
    return np.array(X), np.array(y), label_dict

data_dir = "/content/data/gestures/leapGestRecog"
X, y, label_dict = load_gesture_images(data_dir)
print("Loaded:", X.shape, y.shape)
print("Classes:", label_dict)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear', verbose=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

inv_label_dict = {v: k for k, v in label_dict.items()}

def show_gesture_predictions(model, X_test, y_test, img_size=64):
    indices = np.random.choice(len(X_test), 9)
    plt.figure(figsize=(10, 10))
    for i, idx in enumerate(indices):
        img = X_test[idx].reshape(img_size, img_size)
        true = inv_label_dict[y_test[idx]]
        pred = inv_label_dict[model.predict([X_test[idx]])[0]]
        plt.subplot(3, 3, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f"True: {true}\nPred: {pred}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_gesture_predictions(model, X_test, y_test)
